# Pipeline to run data on trained model

## Parameters

In [1]:
model_artifact = "model-mtzxw7nb:v0"
artifact_dir = './artifacts/model-mtzxw7nb:v0'
window = -1
ver=1

climate_model = 'CESM'
num_input_time_steps = 3 
lead_time = 3
all_start_date = '0400-01-01'
all_end_date = '2021-12-31'

## Download best model

In [ ]:
import wandb
run = wandb.init(project="CI2022_Hackathon", group='CI2022_best_version', job_type='testing')
artifact = run.use_artifact('helgemohn/CI2022_Hackathon/'+model_artifact, type='model')
artifact_dir = artifact.download()
run.finish()

## Load best model

In [2]:
# ------------
# hparams
# ------------

PATH = artifact_dir + '/model.ckpt'

from tools.model_siren import DeepNeuralNet
from tools.model_dataloader import CI2022_DataModule
from tools.model_args import get_args
hparams = get_args(DeepNeuralNet, CI2022_DataModule, jupyter_flag=True)

from pytorch_lightning import seed_everything
seed_everything(hparams.seed)  

hparams.gpus = 7
hparams.window = window
hparams.version = ver
hparams.data_path_train = hparams.data_path_train.format(hparams.window, hparams.version)
hparams.data_path_test = hparams.data_path_test.format(hparams.window, hparams.version)
hparams.data_path_meanstddev = hparams.data_path_meanstddev.format(hparams.window, hparams.version)
print(hparams.data_path_train)

# ------------
# data
# ------------
data = CI2022_DataModule(hparams)
hparams.dim_input = data.dim_input
hparams.dim_output = data.dim_output

# ------------
# model
# ------------
model = DeepNeuralNet(hparams)

import torch
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['state_dict'])
epoch = checkpoint['epoch']

model.mean_dox = data.mean[-1]
model.std_dox = data.std[-1]
model.eval()


Global seed set to 1


./data/helge_TRAIN_dataset_window-1_sincostime_ver1.nc


DeepNeuralNet(
  (model): Sequential(
    (0): Siren(
      (activation): Sine()
    )
    (1): Siren(
      (activation): Sine()
    )
    (2): Siren(
      (activation): Sine()
    )
    (3): Siren(
      (activation): Sine()
    )
    (4): Siren(
      (activation): Sine()
    )
    (5): Siren(
      (activation): Identity()
    )
  )
)

In [3]:
%load_ext autoreload
%autoreload 2
# ------------
# ready to start testing
# ------------
from tools.model_testing import test
prediction = test(model, data, hparams)
prediction

wandb: Currently logged in as: helgemohn (use `wandb login --relogin` to force relogin)


Auto select gpus: [7]
/atm_chemmod/swift/swift_nn/miniconda3/envs/swift-ai/lib/python3.8/site-packages/pytorch_lightning/loops/utilities.py:91: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]


Data loaded (test 1.01e+06)


Predicting: 0it [00:00, ?it/s]

[{'predict_loss': array([[-0.00882804],
       [-0.3614623 ],
       [ 2.3192825 ],
       ...,
       [-1.3302052 ],
       [ 0.28146714],
       [ 0.48137465]], dtype=float32)}]


[{'predict_loss': array([[-0.00882804],
         [-0.3614623 ],
         [ 2.3192825 ],
         ...,
         [-1.3302052 ],
         [ 0.28146714],
         [ 0.48137465]], dtype=float32)}]

In [5]:
len(prediction[0]['predict_loss'])

1012242

## CI2022 Hackathon DATA

In [1]:
import numpy as np
test_input = np.load('./data/test_input.npy')
print(test_input.shape)


(120, 13, 20, 3)


In [4]:

#make sure we have floats in there
X = test_input.astype(np.float32)

In [ ]:
%load_ext autoreload
%autoreload 2
from tools.data_creation import data_creation
data_creation(data_set, window, ver, all_start_date, all_end_date, lead_time, climate_model, num_input_time_steps)


## store mean stddev

In [ ]:
%load_ext autoreload
%autoreload 2
from tools.data_meanstd import data_meanstd
data_meanstd(window, ver)

